# 1. Configuration & Imports

In [ ]:
# 1. Remove old folder if it exists
!rm -rf YourRepoName

# 2. Clone your code
!git clone https://github.com/mazennh/Gesture-Classification.git

# 3. Install dependencies
!pip install -r /kaggle/working/Gesture-Classification/requirements.txt --quiet

# 4. Allow Python to find your files
import sys
import os
sys.path.append(os.path.abspath("/kaggle/working/Gesture-Classification"))

# 5. Auto-reload (Optional: helps if you edit code and git pull)
%load_ext autoreload
%autoreload 2

print("Setup Complete! Your custom modules are ready.")

In [ ]:
import torch
from torch import nn
import subprocess
from torchinfo import summary
import warnings
from torch.optim.lr_scheduler import StepLR
from pyngrok import ngrok
import data_utils
import model_utils
import train_utils
import vis_utils

warnings.filterwarnings("ignore")
device = "cuda" if torch.cuda.is_available() else "cpu"
device

# 2. Data Pipeline Construction

## 2.1 Dataset Filtering & Splitting

In [ ]:
data_utils.filter_data(input_path = '/kaggle/input/hagrid-classification-512p/hagrid-classification-512p',
           output_path = '/kaggle/working/filtered_data',
           split_path = '/kaggle/working/splited_data',
           classes_list = ["stop", "dislike", "like",
                           "peace", "peace_inverted", "ok",
                           "call", "mute", "stop_inverted"],
           split_ratio = (0.8,0.1,0.1),
           seed = 42
          )

## 2.2 Exploratory Data Analysis (EDA)

### 2.2.1 Class Distribution Analysis

the dataset is well-balanced across all target classes. Since there is no significant class imbalance, no oversampling or class-weighting techniques are required

In [ ]:
data_utils.class_distribution(root_path = '/kaggle/working/filtered_data')

In [ ]:
data_utils.class_distribution(root_path = '/kaggle/working/splited_data/train')

In [ ]:
data_utils.class_distribution(root_path = '/kaggle/working/splited_data/val')

In [ ]:
data_utils.class_distribution(root_path = '/kaggle/working/splited_data/test')

### 2.2.2 Sample Visualization & Resolution Check
Analysis of random samples indicates that most images have a resolution of **512×512**. However, our target architectures **(VGG16, ResNet, InceptionV1, ViT)** generally are optimized for input dimensions of **224×224**.

In [ ]:
vis_utils.visualize_random_samples(root_path = "/kaggle/working/filtered_data",
                                   n_samples=10,
                                   cols = 4)

## 2.3 Image Augmentation & Preprocessing

### Train
* RandomResizedCrop **(Resize -> 224,224)**
* RandomHorizontalFlip
* RandomRotation **(5° to 15°)**
* Brightness/Contrast **(ColorJitter)**
* ToTensor
* Normalize

### Test & Val

* Turn data into **tensors**
* Normalization
* Resize **(224,224)** 

In [ ]:
dls = data_utils.create_dataloaders(
    data_dir="/kaggle/working/splited_data",
    batch_size=32,
    img_size=224
)

train_dataloader, val_dataloader, test_dataloader, train_dataset, class_names, class_dict = dls


## 2.4 Data Verification

#### Sanity Check

Before feeding data into the model, we perform a final sanity check. We retrieve a single batch from the DataLoader, reverse the normalization, and visualize the images. This ensures that our augmentation pipeline is functioning correctly and that labels match the image content.

In [ ]:
img, _ = next(iter(train_dataloader))
img.shape

In [ ]:
vis_utils.data_verification(dataset = train_dataset,
                            class_names = class_names,
                            n_rows=3,
                            n_cols=4)

# 3. Model

## 3.1 Making Model

In [ ]:
model, architecture_name = model_utils.get_model(num_classes=len(class_names),
                                                 model_name='resnet',
                                                 device=device)

## 3.2 Model Summary

In [ ]:
img, _ = next(iter(train_dataloader))
img.shape

In [ ]:
summary(model,input_size=(img.shape))

# 4 Model Training

## 4.1 Loss Function & Optimizer

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)

## 4.2 Model Training

In [ ]:
model_train=train_utils.train(model=model,
                              train_dataloader=train_dataloader,
                              val_dataloader=val_dataloader,
                              optimizer=optimizer,
                              loss_fn=loss_fn,
                              num_classes = len(class_names),
                              best_model = architecture_name,
                              scheduler=scheduler,
                              device=device,
                              patience=10,
                              experiment_name = architecture_name,
                              epochs=100)